# Read ASC files

## Import libraries

In [14]:
import glob, os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import rasterio as rio
import gdal

ModuleNotFoundError: No module named 'gdal'

## Read DENUE file

In [13]:
gdf = gpd.read_file('data/colima/agebs.gpkg')
gdf = gdf[["CVE_AGEB", "geometry"]]
gdf.head()

,CVE_AGEB,geometry
0,0676,"POLYGON ((2319182.093 807954.424, 2319286.446 ..."
1,0727,"POLYGON ((2318544.164 807545.878, 2318566.020 ..."
2,1142,"POLYGON ((2316882.335 805042.325, 2317096.142 ..."
3,0411,"POLYGON ((2319801.226 808675.655, 2319836.062 ..."
4,1068,"POLYGON ((2320222.167 809638.735, 2320222.724 ..."


## GDAL

## Ubicación de los archivos ASC

In [6]:
asc_files = glob.glob('data/Rasters/*/*.asc')
asc_files[:5]

['data/Rasters\\Hydraulic\\Depth____0.0.asc',
 'data/Rasters\\Hydraulic\\Depth____10200.0.asc',
 'data/Rasters\\Hydraulic\\Depth____10500.0.asc',
 'data/Rasters\\Hydraulic\\Depth____10800.0.asc',
 'data/Rasters\\Hydraulic\\Depth____11100.0.asc']

## GDAL

In [ ]:
# Open the ASC file as a GDAL dataset
asc_ds = gdal.Open("file.asc")

In [ ]:
# Import libraries
import gdal
import geopandas as gpd

# Open the ASC file as a GDAL dataset
asc_ds = gdal.Open("file.asc")

# Get the geotransform and projection of the ASC file
asc_gt = asc_ds.GetGeoTransform()
asc_proj = asc_ds.GetProjection()

# Read the ASC file as a numpy array
asc_arr = asc_ds.ReadAsArray()

# Close the GDAL dataset
asc_ds = None

# Create a GeoDataFrame from the ASC array, geotransform and projection
asc_gdf = gpd.GeoDataFrame.from_features(
    gdal.polygonize(asc_arr, None, options=["8CONNECTED=8"]),
    crs=asc_proj
)

# Set the column name of the ASC values
asc_gdf.columns = ["value", "geometry"]

# Open the gpkg file as a GeoDataFrame
gpkg_gdf = gpd.read_file("file.gpkg")

# Perform a spatial join to extract the zones that intersect with the ASC file
zones_gdf = gpd.sjoin(asc_gdf, gpkg_gdf, how="inner", op="intersects")

# Save the zones as a new gpkg file
zones_gdf.to_file("zones.gpkg", driver="GPKG")